# <center> <img src="../../img/ITESOLogo.png" alt="ITESO" width="480" height="130"> </center>
# <center> **Departamento de Electrónica, Sistemas e Informática** </center>
---
## <center> Computer Systems Engineering  </center>
---
### <center> Big Data Processing </center>
---
#### <center> **Autumn 2025** </center>
---

**Lab 07**: Structured Streaming with Files 

**Date**: October 10th 2025

**Student Name**: Luis Antonio Pelayo Sierra

**Professor**: Pablo Camarillo Ramirez

In [1]:
import findspark
findspark.init()

from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("Structured Streaming with Files") \
    .master("spark://spark-master:7077") \
    .config("spark.ui.port", "4040") \
    .getOrCreate()

sc = spark.sparkContext
sc.setLogLevel("ERROR")

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/10/10 02:06:34 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
!pwd

/opt/spark/work-dir/labs/lab07


In [6]:
!ls ../../

artifacts  datacar_service  img   labs	libs  spark
data	   examples	    jars  lib	opt


In [4]:
# import sys, subprocess
# subprocess.Popen([sys.executable, "../../lib/luis_pelayo/logs.py"])

In [5]:
from luis_pelayo.spark_utils import SparkUtils
from pyspark.sql.functions import split, col

schema = SparkUtils.generate_schema([("Timestamp", "timestamp"),
                                        ("Level", "string"),
                                        ("Message", "string"),
                                        ("Host", "string")])

logs = spark.readStream \
    .format("text") \
    .load("/opt/spark/work-dir/data/logs/")  

logs_parsed = logs.select(split(col("value"), "\\s*\\|\\s*").alias("fields")).select(col("fields").getItem(0).alias("Timestamp").cast("timestamp"),
                                                                                col("fields").getItem(1).alias("Level"),
                                                                                col("fields").getItem(2).alias("Message"),
                                                                                col("fields").getItem(3).alias("Host"))

logs_filtered = logs_parsed.filter((col("Level").isin("ERROR", "CRITICAL")))

query = logs_filtered.writeStream \
                        .outputMode("append") \
                        .format("console") \
                        .option("truncate", False) \
                        .start()

query.awaitTermination(20)

-------------------------------------------
Batch: 0
-------------------------------------------
+-------------------+-----+-----------------------------------+------------+
|Timestamp          |Level|Message                            |Host        |
+-------------------+-----+-----------------------------------+------------+
|2025-10-10 01:59:19|ERROR|503 Service Unavailable            |payment-node|
|2025-10-10 01:59:19|ERROR|503 Service Unavailable            |auth-service|
|2025-10-10 02:06:54|ERROR|502 Bad Gateway                    |cache-proxy |
|2025-10-10 01:59:10|ERROR|Uncaught exception in worker thread|db-cluster  |
|2025-10-10 01:58:38|ERROR|Timeout contacting upstream service|db-cluster  |
+-------------------+-----+-----------------------------------+------------+

-------------------------------------------
Batch: 1
-------------------------------------------
+-------------------+--------+----------------------------+------------+
|Timestamp          |Level   |Message  

False

-------------------------------------------
Batch: 3
-------------------------------------------
+-------------------+-----+--------------------+------------+
|Timestamp          |Level|Message             |Host        |
+-------------------+-----+--------------------+------------+
|2025-10-10 02:07:22|ERROR|Transaction rollback|auth-service|
|2025-10-10 02:07:22|ERROR|Transaction rollback|auth-service|
+-------------------+-----+--------------------+------------+

-------------------------------------------
Batch: 4
-------------------------------------------
+-------------------+-----+-----------------------------------+------------+
|Timestamp          |Level|Message                            |Host        |
+-------------------+-----+-----------------------------------+------------+
|2025-10-10 02:07:32|ERROR|Uncaught exception in worker thread|auth-service|
+-------------------+-----+-----------------------------------+------------+

-------------------------------------------
Bat

In [7]:
sc.stop()